# Curve fitting in python
## Computational Physics - Phy 325
An introduction to various curve fitting routines useful for physics work.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

First, define `x` as a linear space with 100 points that range from 0 to 10.

In [ ]:
x = np.linspace(0,10,100)

`y` is mock data that we create by linear function with a slope of 1.45. We also add a small amount of random data to simulate noise as if this were a measured quantity. Notice the `np.random.random` function takes a size argument and returns an array of random numbers.

In [ ]:
y = 1.45 * x + 5*np.random.random(len(x))

In [ ]:
plt.plot(x,y,".")

The data is pretty clearly linear, but we can fit a line to determine the slope. A 1st order polynomial is a line, so we use `polyfit`:

In [ ]:
# execute the fit on the data; a 1-dim fit (line)
fit, cov = np.polyfit(x, y, 1,cov=True)  

In [ ]:
cov

The fit is stored in a variable called `fit` which has two elements. The covariance matrix diagonals are the variance in the fit parameters. Use these to calculate the standard deviation in the fit parameters:

In [ ]:
a = fit[0]
b = fit[1]
print("a =",a," b =",b)

In [ ]:
da = np.sqrt(cov[0,0])
db = np.sqrt(cov[1,1])
print("da = {:.3f}   db = {:.3f}".format(da,db))  #note this method to format printing.  More on this soon

Now plot the data and the fits with their full range of values (a±da and b±db)

In [ ]:
plt.plot(x,y,".")
plt.plot(x,(a+da)*x + b,"r")
plt.plot(x,(a-da)*x + b,"r")
plt.plot(x,a*x + b + db,"b")
plt.plot(x,a*x + b - db,"b")

## Using linear regression builtins

In [ ]:
from scipy.stats import linregress

In [ ]:
x = np.linspace(0,10,100)
y = 0.01 * x + 2 + 2*np.random.random(len(x))

In [ ]:
plt.plot(x,y,".")
plt.ylim(0,10)

In [ ]:
regresfit = linregress(x,y)
regresfit

## General function fitting
### For more than just polynomials
> "When choosing a fit, Polynomial is almost always the wrong answer" - Ancient Physics Wisdom

Often there is a better model that describes the data. In most cases this is a known function; something like a power law or an exponential. In these cases, there are two options:
1. Convert the variables so that a plot will be linear (i.e. plot the `log` of your data, or the square root, or the square etc.). This is highly effective because a linear fit can often be more accurate than a fit of another function.
2. Perform a nonlinear fit to the function that models your data. We'll illustrate this below and show how even a "decent" fit gives several % error.

First, we import the functions that do nonlinear fitting:

In [ ]:
from scipy.optimize import curve_fit

Then define a function that we expect models our system. In this case, exponential decay with an offset.

In [ ]:
def func(x, a, b, c):
    return a * np.exp(-b * x) + c

Create a pure (i.e. exact) set of data with some parameters, and then simulate some data of the same system (by adding random noise).

In [ ]:
y = func(x, 2.5, 0.6, 0.5)
ydata = y * (1.0 + 0.1*np.random.normal(size=len(x)))

In [ ]:
plt.plot(x,ydata,".")

Now carry out the fit. `curve_fit` returns two outputs, the fit parameters, and the covariance matrix. We won't use the covariance matrix yet, but it's good practice to save it into a variable.

In [ ]:
parameters, covariance = curve_fit(func, x, ydata)
parameters  #the fit results for a, b, c

In [ ]:
covariance

We can see the parameters are a reasonable match to the pure function we created above. Next, we want to create a "best fit" data set but using the parameters in the model function `func`. The "splat" operator is handy for this, it unpacks the `parameters` array into function arguments `a`, `b`, and `c`.

In [ ]:
yfit = func(x, *parameters) 
# the splat operator unpacks an array into function arguments

In [ ]:
plt.plot(x,ydata,".")
plt.plot(x,yfit)
plt.plot(x,y)

Looks pretty good as far as fits go. Let's check out the error:

In [ ]:
plt.plot(x,((yfit-y)/y)*100)
plt.title("Fit error %")

To further illustrate the variation in this fit, repeat all the cells (to get new random noise in the data) and you'll see the fit changes. Sometimes, the error is as large as 10%.

## Modeling by rescaling data
### The "fit a line to anything" approach
> "With a small enough data set, you can always fit it to a line" - More Ancient Wisdom

In [ ]:
ylog = np.log(ydata)
plt.plot(x,ylog,".")

Not very linear... why not?

In [ ]:
# because we have a y offset of 0.5. Taking that out gives a linear result.
ylog = np.log(np.abs(ydata - 0.5))
plt.plot(x,ylog,".")

In [ ]:
fitlog = np.polyfit(x, ylog, 1,full=True)

In [ ]:
fitlog

In [ ]:
flog = np.poly1d(fitlog[0])
plt.plot(x,ylog)
plt.plot(x,flog(x))

Now to finally back out the exponential from the linear fit:

In [ ]:
ylogfit = np.exp(flog(x))
plt.plot(x,ylogfit + 0.5)
plt.plot(x,ydata,'.')

The tail is good but the beginning is not. We actually want to emphasize that region (becuase it is where our data is more accurate (larger numbers). Also, we can't know the shift is 0.5 so we have to extract it from the data. We can do this by averaging the last 20 points:

In [ ]:
yshift = np.average(ydata[-20:])
yshift

In [ ]:
# Also, run the fit on the early part of the data to emphasize those points.
ylog = (np.log(ydata[1:25] - yshift))
fitlog = np.polyfit(x[1:25], ylog, 1,full=True)  
flog = np.poly1d(fitlog[0])
plt.plot(x[1:25],ylog)
plt.plot(x[1:25],flog(x[1:25]))

In [ ]:
ylogfit = np.exp(flog(x))
plt.plot(x,ylogfit+yshift)
plt.plot(x,ydata)

Very nice.

In [ ]:
plt.plot(x,((ylogfit+yshift-y)/y)*100)
plt.title("Fit error %")

The builtin fit (scipy.optimize.curve_fit) generally does better, it's pretty well designed for fitting (as you would hope).